In [ ]:
import requests
import pandas as pd


# Questão 1

In [ ]:
r = requests.get(r'https://www.akitaonrails.com/sitemap.xml')
print(r.status_code)
print(r.headers)
xml = r.text


In [4]:
xml[:100]

'<?xml version="1.0" encoding="UTF-8"?>\n<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n'

# Questão 2

In [5]:
r = requests.get(r'https://www.akitaonrails.com/robots.txt')
print(r.status_code)
print(r.headers)
print(r.text)


200
{'Server': 'Cowboy', 'Date': 'Wed, 14 Dec 2022 15:01:42 GMT', 'Connection': 'keep-alive', 'Last-Modified': 'Tue, 25 Oct 2022 13:36:13 GMT', 'Content-Type': 'text/plain', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains', 'Content-Length': '287', 'Via': '1.1 vegur'}
# See http://www.robotstxt.org/wc/norobots.html for documentation on how to use the robots.txt file
#
# To ban all spiders from the entire site uncomment the next two lines:
# User-Agent: *
# Disallow: /
Disallow: /admin
Disallow: /files
Sitemap: http://www.akitaonrails.com/sitemap.xml



# Questão 3

In [7]:
df = pd.read_xml(xml, encoding='utf-8')
df.head()

,loc,priority,lastmod
0,http://www.akitaonrails.com,1.0,None
1,http://www.akitaonrails.com/Off-Topic,1.0,2018-07-27
2,http://www.akitaonrails.com/screencasts,1.0,2015-10-13
3,http://www.akitaonrails.com/2022/12/12/akitand...,0.9,2022-12-12
4,http://www.akitaonrails.com/2022/11/22/akitand...,0.9,2022-11-22


# Questão 4.1

In [8]:
df.dtypes

loc          object
priority    float64
lastmod      object
dtype: object

In [9]:
# print column dtypes
df.lastmod = df.lastmod.astype('datetime64[ns]')
print(df.dtypes)


loc                 object
priority           float64
lastmod     datetime64[ns]
dtype: object


In [10]:
(
    df
    .lastmod
    .groupby(df.lastmod.dt.year)
    .count()
)


lastmod
2015.0    1008
2016.0      39
2017.0      70
2018.0       9
2019.0      64
2020.0      27
2021.0      27
2022.0      23
Name: lastmod, dtype: int64

# Questão 4.2

Primeiro passo é aplicar um split na coluna *loc* pelo caractere ```/```. 
Depois temos que remover as informações que ficaram de forma inválida nessas células para casos onde a URL não tinha uma data.

In [11]:
df = pd.concat([df,
                (df['loc']
                 .str
                 .split('/', expand=True)
                 .loc[:, [3, 4]])],
               axis=1)
df.head(6)


,loc,priority,lastmod,3,4
0,http://www.akitaonrails.com,1.0,NaT,None,None
1,http://www.akitaonrails.com/Off-Topic,1.0,2018-07-27,Off-Topic,None
2,http://www.akitaonrails.com/screencasts,1.0,2015-10-13,screencasts,None
3,http://www.akitaonrails.com/2022/12/12/akitand...,0.9,2022-12-12,2022,12
4,http://www.akitaonrails.com/2022/11/22/akitand...,0.9,2022-11-22,2022,11
5,http://www.akitaonrails.com/2022/11/08/akitand...,0.9,2022-11-08,2022,11


Agora vamos renomear as colunas novas para *year* e *month* e vamos aproveitar para converte-las para numérico. 
Mantendo ```NaN``` em caso de informação não ser numérica.

In [12]:
df.rename(columns={3: 'year', 4: 'month'}, inplace=True)
df = df.assign(year=pd.to_numeric(df['year'], errors='coerce'),
               month=pd.to_numeric(df['month'], errors='coerce'))
df.head(5)


,loc,priority,lastmod,year,month
0,http://www.akitaonrails.com,1.0,NaT,NaN,NaN
1,http://www.akitaonrails.com/Off-Topic,1.0,2018-07-27,NaN,NaN
2,http://www.akitaonrails.com/screencasts,1.0,2015-10-13,NaN,NaN
3,http://www.akitaonrails.com/2022/12/12/akitand...,0.9,2022-12-12,2022.0,12.0
4,http://www.akitaonrails.com/2022/11/22/akitand...,0.9,2022-11-22,2022.0,11.0


# Questão 4.3

In [13]:
df.year.value_counts().sort_index()


2006.0     87
2007.0    215
2008.0    251
2009.0    122
2010.0    110
2011.0     48
2012.0     42
2013.0     90
2014.0     48
2015.0     42
2016.0     38
2017.0     31
2018.0     39
2019.0     37
2020.0     21
2021.0     21
2022.0     23
Name: year, dtype: int64

# Questão 4.4

In [14]:
df.month.value_counts().sort_index()


1.0     111
2.0      73
3.0      68
4.0     106
5.0      94
6.0     100
7.0     101
8.0     122
9.0     125
10.0    126
11.0    127
12.0    112
Name: month, dtype: int64

# Bonus

In [ ]:
(df.groupby(['year', 'month'])
 .aggregate({'loc': 'count'})).to_markdown('published_posts.md')
